# Fase 1: Limpeza e Pré-processamento dos Dados

Este notebook executa a primeira fase do projeto de previsão de churn: a limpeza e o pré-processamento dos dados. As seguintes etapas serão realizadas:

1.  **Carregamento dos Dados**: Importação das bibliotecas e leitura do arquivo CSV.
2.  **Tratamento da Variável Alvo (`Churn`)**: Remoção de registros onde a variável alvo é nula.
3.  **Correção de Tipos e Tratamento de Nulos (`Charges.Total`)**: Conversão da coluna para o tipo numérico e tratamento inteligente dos valores ausentes.
4.  **Validação da Consistência dos Dados**: Verificação de regras de negócio entre as colunas de serviços.
5.  **Salvamento dos Dados Limpos**: Exportação do DataFrame processado para um novo arquivo CSV.

In [1]:
import pandas as pd
import numpy as np

# Desabilitar warnings para uma saída mais limpa (opcional)
import warnings
warnings.filterwarnings('ignore')

### 1. Carregamento dos Dados

In [2]:
# O caminho do arquivo foi fornecido no contexto.
caminho_arquivo = r'E:\Documentos\Cursos\Alura\ONE\Desafio-03\telecom_x.csv'

# Carregar o dataset
df = pd.read_csv(caminho_arquivo)

# Exibir as primeiras linhas e informações gerais
print("Dimensões Originais do Dataset:", df.shape)
print("\nInformações Iniciais:")
df.info()

Dimensões Originais do Dataset: (7267, 21)

Informações Iniciais:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7043 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 

In [9]:
# Removendo a coluna de identificação do cliente, pois não possui valor preditivo.
# O parâmetro errors='ignore' evita que o código falhe caso a célula seja executada mais de uma vez.
df.drop(columns=['customerID'], axis=1, inplace=True, errors='ignore')

print("Coluna 'customerID' removida com sucesso.")
print(f"Dimensões do DataFrame após a remoção: {df.shape}")

Coluna 'customerID' removida com sucesso.
Dimensões do DataFrame após a remoção: (7043, 20)


### 2. Tratamento da Variável Alvo (`Churn`)

A variável alvo é essencial para o treinamento de um modelo supervisionado. Registros com o valor de `Churn` ausente não podem ser utilizados e serão removidos.

In [10]:
print(f"Quantidade de valores nulos em 'Churn' antes: {df['Churn'].isnull().sum()}")

# Remover linhas onde a coluna 'Churn' é nula
df.dropna(subset=['Churn'], inplace=True)

print(f"Quantidade de valores nulos em 'Churn' depois: {df['Churn'].isnull().sum()}")
print(f"Novas dimensões do Dataset: {df.shape}")

Quantidade de valores nulos em 'Churn' antes: 0
Quantidade de valores nulos em 'Churn' depois: 0
Novas dimensões do Dataset: (7043, 20)


### 3. Correção de Tipos e Tratamento de Nulos (`Charges.Total`)

A coluna `Charges.Total` frequentemente é carregada como `object` (texto) se contiver espaços ou outros caracteres não numéricos. O primeiro passo é convertê-la para um formato numérico, forçando os valores inválidos a se tornarem `NaN` (Not a Number).

In [11]:
# Converter 'Charges.Total' para numérico. errors='coerce' transforma o que não for número em NaN.
df['Charges.Total'] = pd.to_numeric(df['Charges.Total'], errors='coerce')

# Verificar quantos valores nulos foram criados/encontrados
nulos_total_charges = df['Charges.Total'].isnull().sum()
print(f"Quantidade de valores nulos em 'Charges.Total' após conversão: {nulos_total_charges}")

Quantidade de valores nulos em 'Charges.Total' após conversão: 0


#### 3.1 Investigando a Causa dos Valores Nulos

Antes de aplicar uma regra genérica de imputação, vamos investigar por que esses valores são nulos. A hipótese é que eles pertencem a clientes novos com `tenure` (tempo de contrato) igual a 0.

In [12]:
# Filtrar o dataframe para ver os registros onde 'Charges.Total' é nulo
registros_nulos = df[df['Charges.Total'].isnull()]

print("Valores de 'tenure' para registros com 'Charges.Total' nulo:")
print(registros_nulos['tenure'].value_counts())

Valores de 'tenure' para registros com 'Charges.Total' nulo:
Series([], Name: count, dtype: int64)


#### 3.2 Imputação dos Valores Nulos

A investigação confirma que todos os valores ausentes em `Charges.Total` correspondem a clientes com `tenure = 0`. Portanto, a imputação correta e que reflete a realidade do negócio é preencher esses valores com `0`.

**Nota sobre a Regra de Imputação:** Embora a instrução geral fosse usar amostragem bootstrap para um grande número de nulos, neste caso específico, a investigação revelou uma causa raiz definida. A imputação com '0' é uma representação factual e mais precisa do que uma imputação estatística aleatória.

In [13]:
# Preencher os valores nulos em 'Charges.Total' com 0
df['Charges.Total'].fillna(0, inplace=True)

print(f"Valores nulos em 'Charges.Total' após imputação: {df['Charges.Total'].isnull().sum()}")

Valores nulos em 'Charges.Total' após imputação: 0


### 4. Validação da Consistência dos Dados

Verificamos se há consistência entre os serviços de internet. Por exemplo, se `InternetService` for 'No', então colunas como `OnlineSecurity`, `TechSupport`, etc., devem ter o valor 'No internet service'.

In [14]:
colunas_servicos_internet = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# Filtra o dataframe onde não há serviço de internet
df_sem_internet = df[df['InternetService'] == 'No']

print("Verificando a consistência para clientes sem internet:\n")
for col in colunas_servicos_internet:
    # Se todos os valores forem 'No internet service', o output será 1. Qualquer outro valor indica inconsistência.
    is_consistent = df_sem_internet[col].nunique() == 1 and df_sem_internet[col].unique()[0] == 'No internet service'
    print(f"{col}: {'Consistente' if is_consistent else 'INCONSISTENTE'}")

Verificando a consistência para clientes sem internet:

OnlineSecurity: Consistente
OnlineBackup: Consistente
DeviceProtection: Consistente
TechSupport: Consistente
StreamingTV: Consistente
StreamingMovies: Consistente


### 5. Salvamento dos Dados Limpos

Após a limpeza e o pré-processamento, o DataFrame resultante é salvo em um novo arquivo CSV para ser usado nas próximas fases de análise e modelagem.

In [15]:
caminho_saida = r'E:\Documentos\Cursos\Alura\ONE\Desafio-03\telecom_x_cleaned.csv'

df.to_csv(caminho_saida, index=False)

print(f"\nDataset limpo e salvo com sucesso em: {caminho_saida}")
print(f"Dimensões finais do Dataset: {df.shape}")


Dataset limpo e salvo com sucesso em: E:\Documentos\Cursos\Alura\ONE\Desafio-03\telecom_x_cleaned.csv
Dimensões finais do Dataset: (7043, 20)


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0 to 7266
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Churn             7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 
 17  